# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from api_keys import g_key

import codecs


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
 

weather_df = pd.read_csv('cities.csv')
weather_df.head()

,city_id,city_name,latitude,longitude,date,temp,humidity,cloudiness,wind
0,2126682,Bilibino,68.05,166.44,08/11/20,38.32,82.0,85.0,1.05
1,3652764,Puerto Ayora,-0.74,-90.35,08/11/20,70.02,73.0,75.0,8.05
2,3683508,El Peñol,1.46,-77.45,08/11/20,75.20,50.0,40.0,6.93
3,3366880,Hermanus,-34.42,19.23,08/11/20,54.00,84.0,48.0,8.99
4,145531,Saurimo,-9.66,20.39,08/11/20,78.01,18.0,0.0,3.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["latitude", "longitude"]]

# Store Humidity in humidity
humidity = weather_df["humidity"]


In [4]:
#Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_cities_df = weather_df.loc[(weather_df["wind"] <= 10) & (weather_df["cloudiness"] == 0) & \
                                   (weather_df["temp"] >= 70) & (weather_df["temp"] <= 80)].dropna()

narrowed_cities_df

,city_id,city_name,latitude,longitude,date,temp,humidity,cloudiness,wind
4,145531,Saurimo,-9.66,20.39,08/11/20,78.01,18.0,0.0,3.31
10,2692969,Malmo,55.61,13.00,08/11/20,72.39,64.0,0.0,6.93
43,1513087,Nurota,40.56,65.69,08/11/20,78.80,21.0,0.0,4.70
52,2654728,Bridlington,54.08,-0.19,08/11/20,74.61,61.0,0.0,9.17
59,601734,Daşoguz,41.84,59.97,08/11/20,78.80,41.0,0.0,6.93
75,3466704,Castro,-24.79,-50.01,08/11/20,76.15,38.0,0.0,7.18
83,5391791,San Clemente,33.43,-117.61,08/11/20,70.23,83.0,0.0,3.00
201,1513491,Yangi Marg`ilon,40.43,71.72,08/11/20,78.80,50.0,0.0,4.70
263,145724,Lucapa,-8.42,20.74,08/11/20,79.54,18.0,0.0,3.18
339,933734,Gweta,-20.18,25.23,08/11/20,72.70,20.0,0.0,8.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_cities_df.loc[:,["city_name", "latitude", "longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["hotel name"] = ""

# Display the result
hotel_df

,city_name,latitude,longitude,hotel name
4,Saurimo,-9.66,20.39,
10,Malmo,55.61,13.00,
43,Nurota,40.56,65.69,
52,Bridlington,54.08,-0.19,
59,Daşoguz,41.84,59.97,
75,Castro,-24.79,-50.01,
83,San Clemente,33.43,-117.61,
201,Yangi Marg`ilon,40.43,71.72,
263,Lucapa,-8.42,20.74,
339,Gweta,-20.18,25.23,


In [7]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city_name"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "hotel name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)
    
# Print end of search once searching is completed
print("-------End of Search-------")


Retrieving Results for Index 4: Saurimo.
Closest hotel in Saurimo is iu Hotel Saurimo Rotunda.
------------
Retrieving Results for Index 10: Malmo.
Closest hotel in Malmo is Grand Circus Hotel.
------------
Retrieving Results for Index 43: Nurota.
Closest hotel in Nurota is Guest House Ruslan Nurata & Tours.
------------
Retrieving Results for Index 52: Bridlington.
Closest hotel in Bridlington is Expanse Bridlington Seafront Hotel.
------------
Retrieving Results for Index 59: Daşoguz.
Closest hotel in Daşoguz is Hotel Şabat.
------------
Retrieving Results for Index 75: Castro.
Closest hotel in Castro is HOTEL ALVORADA.
------------
Retrieving Results for Index 83: San Clemente.
Closest hotel in San Clemente is Holiday Inn Express San Clemente N - Beach Area.
------------
Retrieving Results for Index 201: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is ATLAS HOTEL.
------------
Retrieving Results for Index 263: Lucapa.
Missing field/result... skipping.
------------
Retrieving Re

In [9]:
hotel_df

,city_name,latitude,longitude,hotel name
4,Saurimo,-9.66,20.39,iu Hotel Saurimo Rotunda
10,Malmo,55.61,13.00,Grand Circus Hotel
43,Nurota,40.56,65.69,Guest House Ruslan Nurata & Tours
52,Bridlington,54.08,-0.19,Expanse Bridlington Seafront Hotel
59,Daşoguz,41.84,59.97,Hotel Şabat
75,Castro,-24.79,-50.01,HOTEL ALVORADA
83,San Clemente,33.43,-117.61,Holiday Inn Express San Clemente N - Beach Area
201,Yangi Marg`ilon,40.43,71.72,ATLAS HOTEL
263,Lucapa,-8.42,20.74,
339,Gweta,-20.18,25.23,Gweta Lodge


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city_name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [11]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…